# Deep Learning

### Loading Libraries

In [ ]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
from matplotlib .pyplot import subplots

# Scikit-Learn
from sklearn. pipeline import Pipeline
from sklearn. model_selection import KFold
from sklearn. preprocessing import StandardScaler
from sklearn. model_selection import (train_test_split, GridSearchCV)
from sklearn. linear_model import (LinearRegression, LogisticRegression, Lasso)

# ISLP
from ISLP import load_data
from ISLP.models import ModelSpec as MS

# JavaScript Object Notation
import json

# Glob
from glob import glob

#### Torch-Specific Imports

In [ ]:
import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset

In [ ]:
from torchinfo import summary
from torchvision.io import read_image
from torchmetrics import (MeanAbsoluteError, R2Score)

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

In [ ]:
# from pytorch_lightning.utilities.seed import seed_everything

from pytorch_lightning import seed_everything

seed_everything(0, workers=True)

torch.use_deterministic_algorithms(True, warn_only =True)

In [ ]:
from torchvision.datasets import MNIST, CIFAR100
from torchvision.models import (resnet50, ResNet50_Weights)
from torchvision.transforms import (Resize, Normalize, CenterCrop,ToTensor)

In [ ]:
from ISLP.torch import (SimpleDataModule, SimpleModule, ErrorTracker, rec_num_workers)

In [ ]:
from ISLP.torch.imdb import (load_lookup, load_tensor, load_sparse, load_sequential)

### Single Layer Network on Hitters Data

In [ ]:
Hitters = load_data('Hitters').dropna()

n = Hitters.shape [0]

In [ ]:
model = MS(Hitters.columns.drop('Salary'), intercept =False)

X = model.fit_transform(Hitters).to_numpy()
Y = Hitters['Salary']. to_numpy ()

In [ ]:
(X_train,
 X_test,
 Y_train,
 Y_test) = train_test_split(X,
                            Y,
                            test_size=1/3,
                            random_state=1)

#### Linear Models

In [ ]:
hit_lm = LinearRegression().fit(X_train, Y_train)
Yhat_test = hit_lm.predict(X_test)
np.abs( Yhat_test - Y_test).mean()

In [ ]:
scaler = StandardScaler(with_mean=True, with_std=True)
lasso = Lasso(warm_start=True , max_iter=30000)

standard_lasso = Pipeline(steps=[('scaler', scaler),
                                  ('lasso', lasso)])

In [ ]:
X_s = scaler.fit_transform(X_train)
n = X_s.shape[0]

lam_max = np.fabs(X_s.T.dot(Y_train - Y_train.mean ())).max() / n
param_grid = {'alpha': np.exp(np.linspace(0, np.log(0.01), 100))
              * lam_max}

In [ ]:
cv = KFold(10,
           shuffle=True,
           random_state=1)

grid = GridSearchCV(lasso,
                     param_grid,
                    cv=cv,
                    scoring='neg_mean_absolute_error')

grid.fit(X_train, Y_train);

In [ ]:
trained_lasso = grid.best_estimator_

Yhat_test = trained_lasso.predict(X_test)

np.fabs(Yhat_test - Y_test).mean()

#### Specifying a Network: `Classes and Inheritance`

In [ ]:
class HittersModel(nn.Module):
    def __init__(self, input_size):
        super(HittersModel, self).__init__()
        self.flatten = nn.Flatten()
        self. sequential = nn.Sequential(
            nn.Linear(input_size, 50),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear (50, 1))
    
    def forward(self, x):
        x = self.flatten(x)
        return torch.flatten(self.sequential(x))

In [ ]:
hit_model = HittersModel(X.shape[1])

In [ ]:
summary(hit_model,
        input_size=X_train.shape,
        col_names=['input_size',
                   'output_size',
                   'num_params'])

In [ ]:
X_train_t = torch.tensor(X_train.astype(np.float32))
Y_train_t = torch.tensor(Y_train.astype(np.float32))

hit_train = TensorDataset(X_train_t , Y_train_t )

In [ ]:
X_test_t = torch.tensor(X_test.astype(np.float32))
Y_test_t = torch.tensor(Y_test.astype(np.float32))

hit_test = TensorDataset(X_test_t, Y_test_t)

In [ ]:
max_num_workers = rec_num_workers()

In [ ]:
hit_dm = SimpleDataModule(hit_train,
                          hit_test,
                          batch_size=32,
                          num_workers =min(4, max_num_workers),
                          validation = hit_test)

In [ ]:
hit_module = SimpleModule.regression(hit_model,
                                     metrics={'mae': MeanAbsoluteError()})

In [ ]:
hit_logger = CSVLogger('logs', name='hitters')

In [ ]:
hit_trainer = Trainer(deterministic=True,
                      max_epochs=50,
                      log_every_n_steps=5,
                      logger=hit_logger,
                      callbacks=[ErrorTracker()])

hit_trainer .fit(hit_module , datamodule =hit_dm)

In [ ]:
hit_trainer.test(hit_module, datamodule=hit_dm)

In [ ]:
hit_results = pd.read_csv(hit_logger.experiment. metrics_file_path)

In [ ]:
def summary_plot(results,
                 ax,
                 col='loss',
                 valid_legend ='Validation',
                 training_legend ='Training',
                 ylabel='Loss',
                 fontsize =20):
    for (column,
         color,
         label) in zip ([f'train_{col}_epoch',
                         f'valid_{col}'],
                        ['black', 'red'],
                        [training_legend, valid_legend]):
        results.plot(x='epoch',
                     y=column,
                     label=label,
                     marker='o',
                     color=color,
                     ax=ax)
    ax.set_xlabel('Epoch')
    ax.set_ylabel(ylabel)
    return ax

In [ ]:
fig, ax = subplots(1, 1, figsize=(6, 6))

ax = summary_plot(hit_results,
                  ax,
                  col='mae',
                  ylabel='MAE',
                  valid_legend='Validation(=Test)')

ax.set_ylim([0, 400])
ax.set_xticks(np.linspace(0, 50, 11).astype(int));
plt.grid(True)
plt.show()

In [ ]:
hit_model.eval()
preds = hit_module(X_test_t)
torch.abs(Y_test_t - preds).mean()

#### Cleanup

In [ ]:
del(Hitters,
    hit_model, 
    hit_dm,
    hit_logger,
    hit_test, hit_train,
    X, Y,
    X_test, X_train,
    Y_test, Y_train,
    X_test_t, Y_test_t,
    hit_trainer, hit_module)

### Multilayer Network on the MNIST Digit Data

In [ ]:
(mnist_train,
 mnist_test) = [MNIST(root='data',
                      train=train,
                      download=True,
                      transform = ToTensor())
                for train in [True, False]]

mnist_train

In [ ]:
mnist_dm = SimpleDataModule(mnist_train,
                            mnist_test,
                            validation=0.2,
                            num_workers=max_num_workers,
                            batch_size =256)

In [ ]:
for idx, (X_, Y_) in enumerate(mnist_dm.train_dataloader()):
    print('X:', X_.shape)
    print('Y:', Y_.shape)
    
    if idx >= 1:
        break

In [ ]:
class MNISTModel(nn.Module):
    def __init__ (self):
        super(MNISTModel, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Dropout(0.4))
        self.layer2 = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3))
        self._forward = nn.Sequential(
            self.layer1,
            self.layer2,
            nn.Linear(128, 10))
    
    def forward(self, x):
        return self._forward(x)

In [ ]:
mnist_model = MNISTModel()

In [ ]:
mnist_model(X_).size()

In [ ]:
summary(mnist_model,
        input_data=X_,
        col_names =['input_size',
                    'output_size',
                    'num_params'])

In [ ]:
mnist_module = SimpleModule.classification(mnist_model, num_classes=10)

mnist_logger = CSVLogger('logs', name='MNIST')

In [ ]:
mnist_trainer = Trainer(deterministic=True,
                        max_epochs=30,
                        logger=mnist_logger,
                        callbacks =[ErrorTracker()])

mnist_trainer.fit(mnist_module,
                  datamodule=mnist_dm)

In [ ]:
mnist_results = pd.read_csv(mnist_logger.experiment.metrics_file_path)

fig, ax = subplots(1, 1, figsize=(6, 6))
summary_plot(mnist_results,
             ax,
             col='accuracy',
             ylabel='Accuracy')

ax.set_ylim([0.5, 1])
ax.set_ylabel('Accuracy')
ax.set_xticks(np.linspace(0, 30, 7).astype(int));
plt.grid(True)
plt.show()

In [ ]:
mnist_trainer.test(mnist_module,
                   datamodule=mnist_dm)

In [ ]:
class MNIST_MLR(nn.Module):
    def __init__(self):
        super(MNIST_MLR, self).__init__()
        self.linear = nn.Sequential(nn.Flatten(),
                                    nn.Linear(784, 10))
    def forward(self, x):
        return self.linear(x)

mlr_model = MNIST_MLR()
mlr_module = SimpleModule.classification(mlr_model, num_classes=10)
mlr_logger = CSVLogger('logs', name='MNIST_MLR')

In [ ]:
mlr_trainer = Trainer(deterministic=True,
                      max_epochs=30,
                      callbacks=[ErrorTracker()])

mlr_trainer.fit(mlr_module, datamodule=mnist_dm)

In [ ]:
mlr_trainer .test(mlr_module,
                  datamodule=mnist_dm)

In [ ]:
del(mnist_test,
    mnist_train,
    mnist_model,
    mnist_dm,
    mnist_trainer,
    mnist_module,
    mnist_results,
    mlr_model,
    mlr_module,
    mlr_trainer)

### Convolutional Neural Networks

In [ ]:
(cifar_train,
 cifar_test ) = [ CIFAR100 (root="data",
                                        train=train,
                                        download=True)
                              for train in [True, False]]

In [ ]:
transform = ToTensor()

cifar_train_X = torch.stack([transform(x) for x in
    cifar_train.data])

cifar_test_X = torch.stack([transform(x) for x in
    cifar_test.data])

cifar_train = TensorDataset(cifar_train_X,
                            torch.tensor(cifar_train.targets))

cifar_test = TensorDataset(cifar_test_X,
                           torch.tensor(cifar_test.targets))

In [ ]:
cifar_dm = SimpleDataModule (cifar_train,
                             cifar_test,
                             validation=0.2,
                             num_workers =max_num_workers,
                             batch_size =128)

In [ ]:
for idx, (X_, Y_) in enumerate(cifar_dm.train_dataloader()):
    print('X: ', X_.shape)
    print('Y: ', Y_.shape)
    if idx >= 1:
        break

In [ ]:
fig, axes = subplots(5, 5, figsize=(10, 10))

rng = np.random.default_rng(4)

indices = rng.choice(np.arange(len(cifar_train)), 25,
                     replace=False).reshape((5, 5))
for i in range(5):
    for j in range(5):
        idx = indices[i,j]
        axes[i,j]. imshow(np. transpose(cifar_train[idx][0],
                                        [1, 2, 0]),
                          interpolation =None)
        axes[i,j].set_xticks([])
        axes[i,j].set_yticks([])

In [ ]:
class BuildingBlock(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels):
        super(BuildingBlock, self).__init__()
        self.conv=nn.Conv2d(in_channels=in_channels,
                            out_channels=out_channels,
                            kernel_size=(3, 3),
                            padding='same')
        self.activation=nn.ReLU()
        self.pool=nn.MaxPool2d(kernel_size=(2, 2))
        def forward(self , x):
            return self.pool(self. activation (self.conv(x)))

In [ ]:
class CIFARModel(nn.Module):
    def __init__(self):
        super(CIFARModel, self).__init__()
        sizes=[(3, 32),
               (32, 64),
               (64, 128),
               (128, 256)]
        self.conv=nn.Sequential(*[ BuildingBlock(in_, out_)
                                   for in_, out_ in sizes])
        self.output=nn.Sequential(nn.Dropout(0.5),
                                  nn.Linear (2*2*256, 512),
                                  nn.ReLU(),
                                  nn.Linear(512, 100))
    def forward(self, x):
        val = self.conv(x)
        val = torch.flatten(val, start_dim =1)
        return self.output(val)

In [ ]:
cifar_model = CIFARModel()

summary(cifar_model,
        input_data=X_,
        col_names =['input_size',
                    'output_size',
                    'num_params'])

In [ ]:
cifar_optimizer = RMSprop(cifar_model.parameters(), lr =0.001)

cifar_module = SimpleModule.classification(cifar_model,
                                           optimizer=cifar_optimizer)

cifar_logger = CSVLogger('logs', name='CIFAR100')

In [ ]:
cifar_trainer = Trainer(deterministic=True,
                        max_epochs=30,
                        logger=cifar_logger,
                        callbacks=[ErrorTracker()])

cifar_trainer.fit(cifar_module,
                  datamodule=cifar_dm)

In [ ]:
log_path = cifar_logger.experiment.metrics_file_path

cifar_results = pd.read_csv(log_path)

fig, ax =subplots(1, 1, figsize=(6, 6))
summary_plot(cifar_results,
             ax,
             col='accuracy',
             ylabel='Accuracy')
ax.set_xticks(np.linspace(0, 10, 6).astype(int))
ax.set_ylabel('Accuracy')
ax.set_ylim([0, 1]);

In [ ]:
cifar_trainer.test(cifar_module,
                   datamodule=cifar_dm)

#### Hardware Acceleration

In [ ]:
try:
    for name, metric in cifar_module.metrics.items():
        cifar_module.metrics[name]=metric.to('mps')
    cifar_trainer_mps=Trainer(accelerator='mps',
                              deterministic=True,
                              max_epochs=30)
    cifar_trainer_mps.fit(cifar_module,
                          datamodule=cifar_dm)
    cifar_trainer_mps.test(cifar_module,
                           datamodule=cifar_dm)
except:
    pass

### Using Pretrained CNN Models

In [ ]:
resize = Resize((232, 232))
crop = CenterCrop(224)

normalize = Normalize([0.485, 0.456, 0.406],
                      [0.229, 0.224, 0.225])

imgfiles = sorted([f for f in glob('book_images /*')])
imgs = torch.stack([torch.div(crop(resize(read_image(f))), 255)
                    for f in imgfiles ])

imgs = normalize(imgs)
imgs.size()

In [ ]:
resnet_model = resnet50(weights=ResNet50_Weights.DEFAULT)

summary(resnet_model,
        input_data=imgs,
        col_names =['input_size',
                    'output_size',
                    'num_params'])

In [ ]:
resnet_model.eval()

In [ ]:
img_preds = resnet_model(imgs)

In [ ]:
img_probs = np.exp(np.asarray(img_preds.detach()))

img_probs /= img_probs.sum(1)[:, None]

In [ ]:
labs = json.load(open('imagenet_class_index .json'))

class_labels = pd.DataFrame([(int(k), v[1]) for k, v in
                            labs.items()],
                            columns =['idx', 'label'])

class_labels = class_labels.set_index('idx')
class_labels = class_labels.sort_index()

In [ ]:
for i, imgfile in enumerate(imgfiles):
    img_df = class_labels.copy()
    img_df['prob'] = img_probs[i]
    img_df = img_df.sort_values(by='prob', ascending=False)[:3]
    print(f'Image: {imgfile}')
    print(img_df.reset_index().drop(columns=['idx']))

In [ ]:
del(cifar_test,
    cifar_train,
    cifar_dm,
    cifar_module,
    cifar_logger,
    cifar_optimizer,
    cifar_trainer)

### IMDB Document Classification

In [ ]:
(imdb_seq_train,
 imdb_seq_test) = load_sequential(root='data/IMDB')

padded_sample = np.asarray(imdb_seq_train.tensors[0][0])
sample_review = padded_sample[padded_sample > 0][:12]

sample_review[:12]

In [ ]:
lookup = load_lookup(root='data/IMDB')

' '.join(lookup[i] for i in sample_review)

In [ ]:
max_num_workers=10

(imdb_train,
 imdb_test) = load_tensor(root='data/IMDB')

imdb_dm = SimpleDataModule(imdb_train,
                           imdb_test,
                           validation=2000,
                           num_workers=min(6, max_num_workers),
                           batch_size =512)

In [ ]:
class IMDBModel (nn.Module):
    
    def __init__(self, input_size):
        super(IMDBModel, self).__init__()
        self.dense1 = nn.Linear(input_size , 16)
        self. activation = nn.ReLU()
        self.dense2 = nn.Linear(16, 16)
        self.output = nn.Linear(16, 1)
        
    def forward(self , x):
        val = x
        for _map in[self.dense1,
                    self.activation,
                    self.dense2,
                    self.activation,
                    self.output]:
            val = _map(val)
        return torch.flatten(val)

In [ ]:
imdb_model = IMDBModel(imdb_test.tensors[0].size()[1])
summary(imdb_model,
        input_size = imdb_test.tensors[0].size(),
        col_names =['input_size',
                    'output_size',
                    'num_params'])

In [ ]:
imdb_optimizer = RMSprop(imdb_model.parameters(), lr =0.001)

imdb_module = SimpleModule.binary_classification(
    imdb_model,
    optimizer=imdb_optimizer)

In [ ]:
imdb_logger = CSVLogger('logs', name='IMDB')

imdb_trainer = Trainer(deterministic=True,
                       max_epochs=30,
                       logger=imdb_logger,
                       callbacks=[ErrorTracker()])

imdb_trainer.fit(imdb_module,
                 datamodule =imdb_dm)

In [ ]:
test_results = imdb_trainer.test(imdb_module, datamodule=imdb_dm)

test_results

#### Comparison to Lasso

In [ ]:
((X_train, Y_train),
 (X_valid, Y_valid),
 (X_test, Y_test)) = load_sparse(validation=2000,
                                 random_state =0,
                                 root='data/IMDB')

In [ ]:
lam_max = np.abs(X_train.T * (Y_train - Y_train.mean())).max()

lam_val = lam_max * np.exp(np.linspace(np.log(1),
                                       np.log(1e -4), 50))

In [ ]:
logit = LogisticRegression (penalty='l1',
                            C=1/ lam_max,
                            solver='liblinear',
                            warm_start =True,
                            fit_intercept =True)

In [ ]:
coefs = []

intercepts = []

for l in lam_val:
    logit.C = 1/l
        logit.fit(X_train, Y_train)

coefs.append(logit.coef_.copy())

intercepts.append(logit.intercept_)

In [ ]:
coefs = np.squeeze(coefs)

intercepts = np.squeeze(intercepts)

In [ ]:
%% capture
fig , axes = subplots(1, 2, figsize=(16, 8), sharey=True)

for ((X_, Y_),
     data_,
     color) in zip ([(X_train , Y_train),
                     (X_valid , Y_valid),
                     (X_test , Y_test)],
                    ['Training', 'Validation', 'Test'],
                    ['black', 'red', 'blue']):
    linpred_ = X_ * coefs.T + intercepts[None, :]
    label_ = np.array(linpred_ > 0)
    accuracy_ = np.array([np.mean(Y_ == l) for l in label_.T])
    axes [0]. plot(-np.log(lam_val / X_train.shape[0]),
                   accuracy_,
                   '.--',
                   color=color,
                   markersize=13,
                   linewidth=2,
                   label=data_)
    axes[0].legend()
    axes[0].set_xlabel(r'$-\ log (\ lambda)$', fontsize=20)
    axes[0].set_ylabel('Accuracy', fontsize =20)

In [ ]:
imdb_results = pd.read_csv(imdb_logger.experiment.metrics_file_path)

summary_plot(imdb_results,
             axes[1],
             col='accuracy',
             ylabel='Accuracy')

axes[1].set_xticks(np.linspace(0, 30, 7).astype(int))
axes[1].set_ylabel('Accuracy ', fontsize=20)
axes[1].set_xlabel('Epoch ', fontsize=20)
axes[1].set_ylim([0.5 , 1]);
axes[1].axhline(test_results[0]['test_accuracy'],
                color='blue',
                linestyle='--',
                linewidth=3)
fig

In [ ]:
del(imdb_model,
    imdb_trainer,
    imdb_logger,
    imdb_dm,
    imdb_train,
    imdb_test)

In [ ]:
### Recurrent Neural Networks